In [1]:
import os
import re
import sys
import pymongo

In [3]:
print("current python recursive depth limit is: "+ str(sys.getrecursionlimit()))
sys.setrecursionlimit(13000)
print("new python recursive depth limit is: "+ str(sys.getrecursionlimit()))

current python recursive depth limit is: 13000
new python recursive depth limit is: 13000


In [ ]:
start_path = os.getcwd()
print(start_path)

In [ ]:
client = MongoClient("mongodb://localhost:27017")
db = client.Reddit
submissions_collection = db['submissions_2016']
comments_collection = db['comments_2016']
nested_comments = db['comments_2016_nested']

In [ ]:
submissions_cursor = submissions_collection.find()


In [4]:
def save_the_children(root_id,source_collection,target_collection,parent_document):
    document = find_the_children(source_collection,target_collection,parent_document,0)
    #print("looking for id: "+ parent_document['id'] + " in "+ str(target_collection))
    target_collection.update_one(
                {"id":root_id},
                { "$push" : { "_child_post": document} },
                upsert=True
            ).matched_count
    return document

In [ ]:
def find_the_children(source_collection,target_collection,parent_document,depth):
    depth = depth +1
    childs_parent_id= "t1_"+parent_document['id']
    
    child_cursor = source_collection.find({"parent_id": childs_parent_id})
    number_of_children = 0
    for child_document in child_cursor:
        number_of_children = number_of_children + 1
        parent_document['_child_post'] = find_the_children(source_collection,target_collection,child_document,depth)
        parent_document['_number_of_children'] = number_of_children
    return parent_document

In [ ]:
#Create nesting collection
number_of_submissions = 0
total_depth_to_end = 0
total_children_count = 0

for document in submissions_cursor:
    number_of_submissions += 1
    updt(true_cursor.count(), number_of_submissions)
    thread_metrics = {}
    link_id = document['name']
    text=document['selftext']
    children_count =0
    depth_to_end = 0
    if ( text == '[removed]' or text == '[deleted]'):
        continue
    else:
        nested_comments.replace_one({"_id": document['_id']},document, upsert=True) #add post parent
        
        child_cursor = comments_collection.find({"parent_id": document['name']})
        for child_document in child_cursor:
            children_count = children_count + 1
            new_document= save_the_children(document['id'],comments_collection,nested_comments,child_document)
            try:
                childs_depth = new_document['_conversation_depth']
                if(type(childs_depth)==int):
                    if(depth_to_end<childs_depth):
                        depth_to_end=childs_depth
            except KeyError:
                pass            

        nested_comments.update_one(
            {"id": document['id']},
            {"$set":{"_length_of_comment": len(document['selftext'])}}
        )
        
        nested_comments.update_one(
            {"id": document['id']},
            {"$set":{"_number_of_children": children_count}}
        )
        total_children_count = + children_count
        
print("Total number of submissions: " + str(submissions.count()))
print("Average length of children: " + str(total_children_count/number_of_submissions))